In [4]:
import os
import numpy as np
!pip install -q wfdb
import matplotlib.pyplot as plt
import wfdb  # MIT-BIH compatible reader

In [5]:
# dataset constants
NUM_OF_PARTICIPANTS = 43
NUM_OF_SESSIONS = 3
NUM_OF_GESTURES = 16
NUM_OF_TRIALS = 7

# other constants
FS = 2048
WINDOW_SIZE = int(0.6 * FS)   # 1229 samples
STEP_SIZE = WINDOW_SIZE // 2  # 50% overlap

METHODS FOR PRE-PROCESSING AND WINDOWING

In [6]:
from scipy.signal import butter, filtfilt, iirnotch

"""Bandpass filter to keep only EMG band."""
def bandpass_filter(data, low=20, high=450, fs=2048, order=4):
    nyq = fs / 2
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    return filtfilt(b, a, data, axis=0)

"""Notch filter to remove powerline noise (60Hz)."""
def notch_filter(data, freq=60.0, fs=2048, quality=30):
    """Notch filter to remove powerline noise (60 Hz)."""
    nyq = fs / 2
    w0 = freq / nyq
    b, a = iirnotch(w0, quality)
    return filtfilt(b, a, data, axis=0)

"""High-pass filter to remove DC offset / drift."""
def dc_removal(data, cutoff=0.1, fs=2048, order=2):
    nyq = fs / 2
    b, a = butter(order, cutoff / nyq, btype='high')
    return filtfilt(b, a, data, axis=0)

"""Apply DC removal + 60Hz notch + bandpass in order."""
def full_preprocess(data, fs=2048):
    x = dc_removal(data, fs=fs)
    x = notch_filter(x, fs=fs)
    x = bandpass_filter(x, fs=fs)
    return x

"""Segment the data into 600ms windows."""
def sliding_window(arr, window_size, step_size):
    for start in range(0, len(arr) - window_size + 1, step_size):
        end = start + window_size
        yield start, end

PRE-PROCESSING + WINDOWING AND CONVERTING EMG DATA

In [10]:
mainFolder = r".\\Grabmyo-1.0.2"
converted_root = "Processed_data"
os.makedirs(converted_root, exist_ok=True)  # dir for the processed + windowed data

FOREARM_CHANNELS = list(range(0, 16))
WRIST_CHANNELS = list(range(17, 23)) + list(range(25, 31))

count = 0
for sessionNum in range(1, NUM_OF_SESSIONS+1):
    sessionFolder = f"Session{sessionNum}_Processed"
    sessionPath = os.path.join(converted_root, sessionFolder)
    os.makedirs(sessionPath, exist_ok=True)

    for participantNum in range(1, NUM_OF_PARTICIPANTS+1):
        forearm_windows, forearm_labels, forearm_meta = [], [], []
        wrist_windows, wrist_labels, wrist_meta = [], [], []

        for gestureNum in range(1, NUM_OF_GESTURES+2):
            for trialNum in range(1, NUM_OF_TRIALS+1):
                filename = f"session{sessionNum}_participant{participantNum}_gesture{gestureNum}_trial{trialNum}"
                filepath = os.path.join(mainFolder, f"Session{sessionNum}", f"session{sessionNum}_participant{participantNum}", filename)

                try:
                    record = wfdb.rdrecord(filepath)
                    emgData = record.p_signal
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")
                    continue

                # ---------- Process Forearm ----------
                forearm = emgData[:, FOREARM_CHANNELS]
                forearm_preprocessed = full_preprocess(forearm, fs=FS)
                for start, end in sliding_window(forearm_preprocessed, WINDOW_SIZE, STEP_SIZE):
                    window = forearm_preprocessed[start:end, :]
                    forearm_windows.append(window)
                    forearm_labels.append(gestureNum)
                    forearm_meta.append((sessionNum, participantNum, gestureNum, trialNum))

                # ---------- Process Wrist ----------
                wrist = emgData[:, WRIST_CHANNELS]
                wrist_preprocessed = full_preprocess(wrist, fs=FS)
                for start, end in sliding_window(wrist_preprocessed, WINDOW_SIZE, STEP_SIZE):
                    window = wrist_preprocessed[start:end, :]
                    wrist_windows.append(window)
                    wrist_labels.append(gestureNum)
                    wrist_meta.append((sessionNum, participantNum, gestureNum, trialNum))

        # ---------- Save ----------
        save_path = os.path.join(sessionPath, f"session{sessionNum}_participant{participantNum}.npz")
        np.savez_compressed(save_path,
            forearm_windows=np.array(forearm_windows, dtype=np.float32),
            forearm_labels=np.array(forearm_labels, dtype=np.int32),
            forearm_meta=np.array(forearm_meta, dtype=np.int32),
            wrist_windows=np.array(wrist_windows, dtype=np.float32),
            wrist_labels=np.array(wrist_labels, dtype=np.int32),
            wrist_meta=np.array(wrist_meta, dtype=np.int32)
        )
        
        count += 1
        print(f"Processed: {count} of {NUM_OF_PARTICIPANTS * NUM_OF_SESSIONS}")

print("Pre-processing + windowing complete.")

Processed: 1 of 129
Processed: 2 of 129
Processed: 3 of 129
Processed: 4 of 129
Processed: 5 of 129
Processed: 6 of 129
Processed: 7 of 129
Processed: 8 of 129
Processed: 9 of 129
Processed: 10 of 129
Processed: 11 of 129
Processed: 12 of 129
Processed: 13 of 129
Processed: 14 of 129
Processed: 15 of 129
Processed: 16 of 129
Processed: 17 of 129
Processed: 18 of 129
Processed: 19 of 129
Processed: 20 of 129
Processed: 21 of 129
Processed: 22 of 129
Processed: 23 of 129
Processed: 24 of 129
Processed: 25 of 129
Processed: 26 of 129
Processed: 27 of 129
Processed: 28 of 129
Processed: 29 of 129
Processed: 30 of 129
Processed: 31 of 129
Processed: 32 of 129
Processed: 33 of 129
Processed: 34 of 129
Processed: 35 of 129
Processed: 36 of 129
Processed: 37 of 129
Processed: 38 of 129
Processed: 39 of 129
Processed: 40 of 129
Processed: 41 of 129
Processed: 42 of 129
Processed: 43 of 129
Processed: 44 of 129
Processed: 45 of 129
Processed: 46 of 129
Processed: 47 of 129
Processed: 48 of 129
P

In [28]:
sample = np.load(r".\Processed_data\Session1_Processed\session1_participant36.npz", allow_pickle=True)
print( f"forearm_labels: {sample['forearm_labels'].shape}" )
print( f"forearm_windows: {sample['forearm_windows'].shape}" )
print( f"forearm_meta: {sample['forearm_meta'].shape}" )

forearm_labels: (1785,)
forearm_windows: (1785, 1228, 16)
forearm_meta: (1785, 4)
